<a href="https://colab.research.google.com/github/LuisAdrian13/parcial2/blob/main/punto_3_parcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#data downloaded for google drive
FILEID = "1IC11LrPCZIo_Am5eXP2p2tDAlrGTlPjn"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O datos.zip && rm -rf /tmp/cookies.txt
!unzip -o datos.zip
!dir

In [ ]:
#librerias
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import warnings
from sklearn.metrics import pairwise_distances
import matplotlib
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

#cargar datos
path_ = 'CaractCE.mat'#Case Western Database
dicX = sio.loadmat(path_)

In [ ]:
Xt = dicX['F'] #datos en el tiempo
Fs = 12000 #frecuencia de muestreo
Tl = Xt.shape[1]/Fs #tamaño del segmento
print('Xt shape:',Xt.shape)
print('tiempo [s]', Tl)

Y = dicX['E']
Ytrue = Y[:,2] #clases fallos en los rodamientos

labels_ = ['NOR','IR1_0.007´´','IR1_0.014´´','IR1_0.021´´',
           'IR2_0.007´´','IR2_0.014´´','IR2_0.021´´',
           'BE_0.007´´','BE_0.014´´','BE_0.021´´'
           ] #nombres de las clases

In [ ]:
print(Ytrue.shape) #etique membresia de los datos 10 posibles valores
print(np.unique(Ytrue))

In [ ]:
#partir datos para train y test
Xtrain, Xtest, Ytrain, _ = train_test_split(Xt, Ytrue, test_size=0.3)

print(f"Xtrain shape {Xtrain.shape}, Ytrain shape {Ytrain.shape }Xtest shape {Xtest.shape} ")

In [ ]:
#calcular espectro de Fourier Xtrain
vf = np.fft.rfftfreq(Xtrain.shape[1],1/Fs) #freq vector
Xw = (abs(np.fft.rfft(Xtrain))) # FFT

In [ ]:
# Crear una lista para almacenar los promedios de cada clase
promedios_tiempo = []

# Calcular el promedio en el tiempo para cada clase
for clase in np.unique(Ytrue):
    promedio_clase = np.mean(Xtrain[Ytrain == clase], axis=0)
    promedios_tiempo.append(promedio_clase)

# Graficar el promedio en el tiempo de cada clase
plt.figure(figsize=(12, 6))
for i, promedio_clase in enumerate(promedios_tiempo):
    plt.plot(promedio_clase, label=labels_[i])

plt.title("Promedio en el tiempo de cada clase de fallo")
plt.xlabel("Muestras en el tiempo")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calcular el promedio en la frecuencia para cada clase
promedios_frecuencia = []

for clase in np.unique(Ytrain):
    espectros = Xw[Ytrain == clase]
    promedio_espectro = np.mean(espectros, axis=0)
    promedios_frecuencia.append(promedio_espectro)

# Graficar el promedio en la frecuencia de cada clase
plt.figure(figsize=(12, 6))
for i, promedio_espectro in enumerate(promedios_frecuencia):
    plt.plot(vf, promedio_espectro, label=labels_[i])

plt.title("Promedio en la frecuencia de cada clase de fallo")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#graficar espectro para clases representativas
sca_ = MinMaxScaler()
Xw_ = sca_.fit_transform(Xw.T).T
#red = TSNE(perplexity = 15,n_components=2,random_state=123,learning_rate='auto',init='pca')
red = PCA(n_components=2)
Z = red.fit_transform(Xw_)

plt.scatter(Z[:,0],Z[:,1],c=Ytrain, label='Xtrain')
plt.colorbar()
plt.show()

In [ ]:

# Calcular la media y la desviación estándar de Xw
media = np.mean(Xw)
desviacion_estandar = np.std(Xw)

# Normalizar Xw
Xw_normalizado = (Xw - media) / desviacion_estandar
Xw_normalizado

In [ ]:
#calcular espectro de Fourier de Xtest
vx = np.fft.rfftfreq(Xtest.shape[1],1/Fs) #freq vector
Xwtest = (abs(np.fft.rfft(Xtest))) # FFT
Xwtest.shape

In [ ]:

# Calcular la media y la desviación estándar de Xwtest
media1 = np.mean(Xwtest)
desviacion_estandar1 = np.std(Xwtest)

# Normalizar Xwtest
Xw_normalizado1 = (Xwtest - media1) / desviacion_estandar1
Xw_normalizado1

In [ ]:
from scipy.spatial.distance import cdist
import  numpy as np
#se crea una lista vacia para agrgar los tipos de fallos de Xtest
v=[]

for o in range (len(Xtest)): #recorre toda la logitud de Xtest
  TT= cdist(Xw_normalizado1,Xw_normalizado) #saca la distacia de Xw_normalizado1[i] con cada uno de los datos de Xw_normalizado
  v.append(Ytrain[np.argmin(TT[o,])]) #Agrega los tipos de fallos a la lista v


In [ ]:

numero_de_fallo = np.unique(Ytrue)
def tipo_de_fallo_y_numero(x,y):
    diccionario = {} #diccionario a llenar

    for i in range(len(x)):
        diccionario[x[i]] = y[i] #llenamos el diccionario con una key x[i] y un value y[i]

    return diccionario

resultado = tipo_de_fallo_y_numero(labels_, numero_de_fallo)
print(resultado)

In [ ]:
def obtener_tipo_de_fallo_de_Xtest(diccionario, lista_valores_en_v):
    Fallos_correspondientes = [] #lista a llenar
#se llena la lista con los tipos de fallos en Xtest
    for valor in lista_valores_en_v:
        for key, value in diccionario.items():
            if value == valor:
                Fallos_correspondientes.append(key) #se agrega a la lista la key del diccionario si el value es igual al valor en la lista de v
                break
        else:
            Fallos_correspondientes.append(None)

    return Fallos_correspondientes


resultados = obtener_tipo_de_fallo_de_Xtest(resultado, v)
print(resultados)#se imprime los tipos de fallos en Xtest